In [147]:
# include("01-Multi# module ThermoGas
using ModelingToolkit, Plots, DifferentialEquations, Revise, Unitful, CoolProp,  Logging
using NonlinearSolve, Printf
@variables t
module Gas
    include("01-ThermoGas.jl")
end

module Steam
    include("01-MultiPhase.jl")
end

module Liq
    include("01-Incompressible.jl")
end


Main.Liq

In [151]:
# Liquid to gas
@component function L2G_HeatExchanger(;name, ϵ = 0.95)
    @named A = Gas.ThermoHeatTransfer()
    @named B = Liq.IncompressibleHeaatTransfer()

    ps = @parameters ϵ = ϵ

    @variables Q̇(t)=0.0 Cmin(t)=0.0
    
    eqs = [
        Q̇ ~ ϵ * (A.p.T - B.p.T) * ((A.C < B.C) *  A.C + (A.C >= B.C) * B.C)      #   heat transfer out of A -> B , if A/T > B/T 
        0 ~ A.Q̇ + B.Q̇
        A.Q̇ ~ -Q̇
    ]

    ODESystem(eqs, t, [Q̇], ps; name = name, systems = [A,B], defaults = [ϵ => 0.95] )
end

@component function S2G_HeatExchanger(;name, ϵ = 0.95)
    @named A = Gas.ThermoHeatTransfer()
    @named B = Steam.SteamHeatTransfer()

    # A = substitute(A, Gas.propDict)
    # B = substitute(B, Steam.hydro_prop_dict)
    ps = @parameters ϵ = ϵ

    @variables Q̇(t)=0.0 Cmin(t)=0.0
    
    eqs = [
        Q̇ ~ ϵ * (A.p.T - B.p.T) * ((A.C < B.C) *  A.C + (A.C >= B.C) * B.C)      #   heat transfer out of A -> B , if A/T > B/T 
        0 ~ A.Q̇ + B.Q̇
        A.Q̇ ~ -Q̇
    ]
    ODESystem(eqs, t, [Q̇], ps; name = name, systems = [A,B], defaults = [ϵ => 0.95] )
end

@component function L2S_HeatExchanger(;name, ϵ = 0.95)
    @named A = Liq.IncompressibleHeaatTransfer()
    @named B = Steam.SteamHeatTransfer()

    # A = substitute(A, Gas.propDict)
    # B = substitute(B, Steam.hydro_prop_dict)
    ps = @parameters ϵ = ϵ

    @variables Q̇(t)=0.0 Cmin(t)=0.0
    
    eqs = [
        Q̇ ~ ϵ * (A.p.T - B.p.T) * ((A.C < B.C) *  A.C + (A.C >= B.C) * B.C)      #   heat transfer out of A -> B , if A/T > B/T 
        0 ~ A.Q̇ + B.Q̇
        A.Q̇ ~ -Q̇
    ]
    ODESystem(eqs, t, [Q̇], ps; name = name, systems = [A,B], defaults = [ϵ => 0.95] )
end


L2S_HeatExchanger (generic function with 1 method)

## Gas 2 liquid xh

In [174]:
@named StmRes   = Steam.Reservoir(P = 40)
@named StmValve = Steam.SteamFlowSource(ṁ = 10)

@named HeRes   = Gas.SinglePortReservoir(P = 10, T = 800)  
@named HeValve = Gas.GasFlowSource(Ṁ = 70)

@named hx = S2G_HeatExchanger()

# steam connectinos
connections = vcat(Steam.hydro_connect(StmRes.n,StmValve.p),
                Steam.hydro_connect(StmValve.n,hx.B.p),
                Gas.gas_connect(HeRes.n,HeValve.p),
                Gas.gas_connect(HeValve.n, hx.A.p))

@named SysS = ODESystem(connections,t; name = :SysS, systems = [HeRes,HeValve,StmRes,StmValve,hx])
SysS = substitute(SysS, Gas.propDict)
SysS = substitute(SysS, Steam.hydro_prop_dict)

# @named StmSys = ODESystem(steamconn,t; systems =[StmRes,StmValve,hx.B])
# StmSys = substitute(StmSys,Steam.hydro_prop_dict)
# @named sys = compose(StmSys,GasSys,hx)

simple_mod = structural_simplify(SysS)
# equations(hx)
prob = ODAEProblem(simple_mod,Pair[],(0.0,1.0))

sol = solve(prob)

# Steam.hydro_prop_dict
equations(SysS)

@show sol[hx.B.C]
@show sol[hx.B.p.T]-sol[hx.B.n.T]
@show sol[hx.A.p.T]-sol[hx.A.n.T]
@show sol[hx.A.n.T]

Gas.showsol([HeRes,HeValve,hx.A,hx.B,StmRes,StmValve],sol)


sol[hx.B.C] = [46369.945794689134, 46369.945794689134]
sol[hx.B.p.T] - sol[hx.B.n.T] = [-277.19011522174856, -277.19011522174856]
sol[hx.A.p.T] - sol[hx.A.n.T] = [66.62332511277134, 66.62332511277134]
sol[hx.A.n.T] = [733.3766748872287, 733.3766748872287]
	 HeRes₊n ṁ = -70.00 	 P =  10.00 	 T = 800.00 
	 HeValve₊p ṁ = 70.00 	 P =  10.00 	 T = 800.00 
	 HeValve₊n ṁ = -70.00 	 P =  10.00 	 T = 800.00 


	 hx₊A₊p ṁ = 70.00 	 P =  10.00 	 T = 800.00  Q̇ = -2.42e+07  C = 3.63e+05 
	 hx₊A₊n ṁ = -70.00 	 P =  10.00 	 T = 733.38  Q̇ = -2.42e+07  C = 3.63e+05 
	 hx₊B₊p ṁ = 10.00 	 P =  40.00 	 T = 250.33  Q̇ = 2.42e+07  C = 4.64e+04 
	 hx₊B₊n ṁ = -10.00 	 P =  40.00 	 T = 527.52  Q̇ = 2.42e+07  C = 4.64e+04 
	 StmRes₊n ṁ = -10.00 	 P =  40.00 	 T = 250.33 
	 StmValve₊p ṁ = 10.00 	 P =  40.00 	 T = 250.33 
	 StmValve₊n ṁ = -10.00 	 P =  40.00 	 T = 250.33 


## Gas to incompressible liquid

In [150]:
@named LiqRes   = Liq.SinglePortReservoir(P = 40, T =700)
@named LiqValve = Liq.IncompressibleFixedFlowSource(Ṁ = 120)

@named HeRes    = Gas.SinglePortReservoir(P = 10, T = 300)  
@named HeValve  = Gas.GasFlowSource(Ṁ = 10)

@named hx = L2G_HeatExchanger()

# steam connectinos
connections = vcat(Liq.incompressible_connect(LiqRes.n, LiqValve.p),  
                Liq.incompressible_connect(LiqValve.n,hx.B.p),
                Gas.gas_connect(HeRes.n, HeValve.p),
                Gas.gas_connect(HeValve.n, hx.A.p))

@named SysS = ODESystem(connections,t; name = :SysS, 
                    systems = [HeRes,HeValve,LiqRes,LiqValve,hx])
SysS = substitute(SysS, Gas.propDict)
SysS = substitute(SysS, Liq.propDict)


simple_mod = structural_simplify(SysS)
prob = ODAEProblem(simple_mod,Pair[],(0.0,1.0))

sol = solve(prob)
@show sol[hx.B.C]
@show sol[hx.B.p.T]-sol[hx.B.n.T]
@show sol[hx.A.p.T]-sol[hx.A.n.T]
@show sol[hx.A.n.T]

Gas.showsol([HeRes,HeValve,hx.A,hx.B,LiqRes,LiqValve],sol)




sol[hx.B.C] = [22634.255999999998, 22634.255999999998]
sol[hx.B.p.T] - sol[hx.B.n.T] = [380.0, 380.0]
sol[hx.A.p.T] - sol[hx.A.n.T] = [-165.65903852080123, -165.65903852080123]
sol[hx.A.n.T] = [465.65903852080123, 465.65903852080123]
	 HeRes₊n ṁ = -10.00 	 P =  10.00 	 T = 300.00 
	 HeValve₊p ṁ = 10.00 	 P =  10.00 	 T = 300.00 
	 HeValve₊n ṁ = -10.00 	 P =  10.00 	 T = 300.00 


	 hx₊A₊p ṁ = 10.00 	 P =  10.00 	 T = 300.00  Q̇ = 8.60e+06  C = 5.19e+04 
	 hx₊A₊n ṁ = -10.00 	 P =  10.00 	 T = 465.66  Q̇ = 8.60e+06  C = 5.19e+04 
	 hx₊B₊p ṁ = 120.00 	 P =  40.00 	 T = 700.00  Q̇ = -8.60e+06  C = 2.26e+04 
	 hx₊B₊n ṁ = -120.00 	 P =  40.00 	 T = 320.00  Q̇ = -8.60e+06  C = 2.26e+04 
	 LiqRes₊n ṁ = -120.00 	 P =  40.00 	 T = 700.00 
	 LiqValve₊p ṁ = 120.00 	 P =  40.00 	 T = 700.00 
	 LiqValve₊n ṁ = -120.00 	 P =  40.00 	 T = 700.00 


## Liquid to Steam

In [173]:
@named LiqRes   = Liq.SinglePortReservoir(P = 40, T =900)
@named LiqValve = Liq.IncompressibleFixedFlowSource(Ṁ = 100)

@named StmRes   = Steam.Reservoir(P = 80)
@named StmValve = Steam.SteamFlowSource(ṁ = 5)

@named hx = L2S_HeatExchanger()

# steam connectinos
connections = vcat(Liq.incompressible_connect(LiqRes.n, LiqValve.p),  
                Liq.incompressible_connect(LiqValve.n,hx.A.p),
                Steam.hydro_connect(StmRes.n, StmValve.p),
                Steam.hydro_connect(StmValve.n, hx.B.p))

@named SysS = ODESystem(connections,t; name = :SysS, 
                    systems = [StmRes,StmValve,LiqRes,LiqValve,hx])

SysS = substitute(SysS, Steam.hydro_prop_dict)
SysS = substitute(SysS, Liq.propDict)


simple_mod = structural_simplify(SysS)
prob = ODAEProblem(simple_mod,Pair[],(0.0,1.0))

sol = solve(prob)
@show sol[hx.B.C]
@show sol[hx.B.p.T]-sol[hx.B.n.T]
@show sol[hx.A.p.T]-sol[hx.A.n.T]
@show sol[hx.A.n.T]

Gas.showsol([StmRes,StmValve,hx.A,hx.B,LiqRes,LiqValve],sol)

@show sol[LiqValve.p.cp]
@show sol[StmRes.n.x]
@show sol[StmValve.p.x]
@show sol[hx.B.n.x]

sol[hx.B.C] = [25292.6503885803, 25292.6503885803]
sol[hx.B.p.T] - sol[hx.B.n.T] = [-231.35263396221762, -231.35263396221762]


sol[hx.A.p.T] - sol[hx.A.n.T] = [574.7339647075249, 574.7339647075249]
sol[hx.A.n.T] = [325.26603529247507, 325.26603529247507]


	 StmRes₊n ṁ = -5.00 	 P =  80.00 	 T = 295.02 
	 StmValve₊p ṁ = 5.00 	 P =  80.00 	 T = 295.02 


	 StmValve₊n ṁ = -5.00 	 P =  80.00 	 T = 295.02 
	 hx₊A₊p ṁ = 100.00 	 P =  40.00 	 T = 900.00  Q̇ = -1.07e+07 

 C = 1.87e+04 
	 hx₊A₊n ṁ = -100.00 	 P =  40.00 	 T = 325.27  Q̇ = -1.07e+07  C = 1.87e+04 


	 hx₊B₊p ṁ = 5.00 	 P =  80.00 	 T = 295.02  Q̇ = 1.07e+07  C = 2.53e+04 
	 hx₊B₊n ṁ = -5.00 	 P =  80.00 	 T = 526.37  Q̇ = 1.07e+07  C = 2.53e+04 


	 LiqRes₊n ṁ = -100.00 	 P =  40.00 	 T = 900.00 
	 LiqValve₊p ṁ = 100.00 	 P =  40.00 	 T = 900.00 


	 LiqValve₊n ṁ = -100.00 	 P =  40.00 	 T = 900.00 
sol[LiqValve.p.cp] = [186.7956, 186.7956]
sol[StmRes.n.x] = [0.0, 0.0]


sol[StmValve.p.x] = [0.0, 0.0]
sol[hx.B.n.x] = [1, 1]


2-element Vector{Int64}:
 1
 1